# S&P500 Dataset Cleaning



Link to dataset: https://www.kaggle.com/datasets/awadhi123/finance-data-sp-500

Description: the dataset contains the candle stick data necessary for the S&P500 index from 2 Jan 2000 to 10 Jul 2020

__Note:__ I kept the variable "volume" since I think it might be a relevant feature

In [64]:
import pandas
df = pandas.read_csv("SP500.csv")   #import the data and read as df

In [65]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,931800000
1,2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1009000000
2,2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1085500000
3,2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1092300000
4,2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1225200000


Adjusted close price is not relevant, I dropped it: 

In [66]:
df = df.drop(['Adj Close'], axis = 1)

In [67]:
print("The first workday of 2018 is " + df["Date"][4528])
print("The last workday of 2018 is " + df["Date"][4778])

The first workday of 2018 is 2018-01-02
The last workday of 2018 is 2018-12-31


I dropped all observations outside the above range and reset the indices

In [68]:
#drop unnecessary observations since we only look at 2018's data
df = df[4528:4779].reset_index()

In [69]:
#check the range of dates
df["Date"]

0      2018-01-02
1      2018-01-03
2      2018-01-04
3      2018-01-05
4      2018-01-08
          ...    
246    2018-12-24
247    2018-12-26
248    2018-12-27
249    2018-12-28
250    2018-12-31
Name: Date, Length: 251, dtype: object

In [70]:
df.head()

,index,Date,Open,High,Low,Close,Volume
0,4528,2018-01-02,2683.729980,2695.889893,2682.360107,2695.810059,3367250000
1,4529,2018-01-03,2697.850098,2714.370117,2697.770020,2713.060059,3538660000
2,4530,2018-01-04,2719.310059,2729.290039,2719.070068,2723.989990,3695260000
3,4531,2018-01-05,2731.330078,2743.449951,2727.919922,2743.149902,3236620000
4,4532,2018-01-08,2742.669922,2748.510010,2737.600098,2747.709961,3242650000
